In [12]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

In [13]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [14]:
RDS_HOST = 'tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com'
print(RDS_HOST)

tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com


In [15]:
postgres_driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""

#### Creamos Dimensión de Personas


In [16]:
sql_query = 'SELECT * FROM persona;'
dimPersonas = pd.read_sql(sql_query, postgres_driver)
dimPersonas.head()

,idpersona,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
0,463292416,cliente,Desiree Diaz,Pasaporte,147682912346113,"9238 Owens Land Suite 734 North Gabriel, NH 77058",+1-686-8,wongronald@hotmail.com
1,335015938,proveedor,Krista Allen,NIE,100989073293314,"438 James Fords Mariaport, VA 88487",(457)489,udiaz@hotmail.com
2,562499590,proveedor,Frank Fitzpatrick,Tarjeta Residencia,105173359417346,74414 Stephanie Place Suite 030 Lake Veronicac...,001-897-,rstone@hotmail.com
3,335101958,proveedor,Alicia Smith,NIE,124952882364420,"709 Mary Alley North Nicole, KY 44947",001-996-,eric43@hotmail.com
4,910546963,proveedor,Joshua Garrett,Pasaporte,115498004649989,"063 Gomez Causeway Lake Frank, AK 02159",707.966.,stonejeffery@gmail.com


#### Dimensión de Usuario


In [17]:
sql_query = 'SELECT * FROM usuario;'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario = df_usuario.rename(columns={'nombre': 'nombre_usuario'})
df_usuario.head()

,idusuario,idrol,nombre_usuario,tipo_documento,num_documento,direccion,telefono,email,clave,estado
0,7868418,10004,Luis Warren,Pasaporte,152338746871809,"3080 Travis Freeway Suite 836 Yorkshire, PA 42313",464-312-,lance31@gmail.com,"[b'd', b'9', b'f', b'1', b'5', b'b', b'a', b'9...",0
1,8210435,10004,Kristen Perez,Pasaporte,114070976995330,"854 Rush Ways East Melvinborough, NM 67431",+1-965-6,marcusjones@yahoo.com,"[b'1', b'b', b'1', b'0', b'f', b'f', b'c', b'6...",1
2,6367235,10002,Andrea Carney,DNI,146279382966274,USCGC Campos FPO AE 16169,001-468-,gainesgregory@yahoo.com,"[b'f', b'c', b'c', b'f', b'f', b'9', b'8', b'2...",1
3,1214471,10003,Lori Morrison,NIE,121915984308230,"081 Jill Run Lake Stephanieborough, MT 86103",001-274-,nwilson@hotmail.com,"[b'7', b'7', b'6', b'b', b'7', b'c', b'1', b'9...",1
4,6279175,10001,Jessica Chambers,Pasaporte,126574599972870,"5937 Sylvia Mountain South Shawn, SD 97049",+1-427-3,travismclean@gmail.com,"[b'4', b'1', b'3', b'8', b'b', b'd', b'd', b'7...",0


In [18]:
sql_query = 'SELECT * FROM rol;'
df_rol = pd.read_sql(sql_query, postgres_driver)
df_rol = df_rol.rename(columns={'nombre': 'nombre_rol'})
df_rol.head()

,idrol,nombre_rol,descripcion,estado
0,10001,Administrador,Este rol tiene acceso completo al sistema y pu...,1
1,10002,Moderador,Este rol tiene permisos para moderar contenido...,1
2,10003,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,10004,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
4,10005,Invitado,Este rol tiene acceso limitado y solo puede ve...,1


In [19]:
df_usuario = df_usuario.merge(df_rol, how='inner', on='idrol')
dimUsuario = df_usuario.drop(['idrol'], axis=1)
dimUsuario.head()

,idusuario,nombre_usuario,tipo_documento,num_documento,direccion,telefono,email,clave,estado_x,nombre_rol,descripcion,estado_y
0,7868418,Luis Warren,Pasaporte,152338746871809,"3080 Travis Freeway Suite 836 Yorkshire, PA 42313",464-312-,lance31@gmail.com,"[b'd', b'9', b'f', b'1', b'5', b'b', b'a', b'9...",0,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
1,8210435,Kristen Perez,Pasaporte,114070976995330,"854 Rush Ways East Melvinborough, NM 67431",+1-965-6,marcusjones@yahoo.com,"[b'1', b'b', b'1', b'0', b'f', b'f', b'c', b'6...",1,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
2,7065613,Peter Knight,Pasaporte,107919605393420,"473 Stephanie Pine Suite 932 Phillipsland, PR ...",339.524.,mthomas@gmail.com,"[b'8', b'c', b'6', b'9', b'e', b'5', b'0', b'6...",0,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
3,4169772,Alicia Sims,NIE,197619393306677,"64161 Amber Circle Suite 999 Port Bradfurt, NV...",+1-486-6,christopher47@gmail.com,"[b'2', b'd', b'0', b'7', b'5', b'2', b'9', b'e...",0,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
4,6895673,Matthew Brown,NIE,105199439972415,"271 Timothy Crossing North Joseph, VA 16705",(268)240,jameswells@gmail.com,"[b'b', b'b', b'd', b'8', b'6', b'd', b'0', b'7...",1,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1


#### Dimensión Artículo


In [20]:
sql_query = 'SELECT * FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo = df_articulo.rename(columns={'nombre': 'nombre_articulo', 'descripcion': 'descripcion_articulo', 'estado': 'estado_articulo'})
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre_articulo,precio_venta,stock,descripcion_articulo,imagen,estado_articulo
0,403456,39582,wkzOzRUc,Vestido Mrs,420.41,48,Artículo Vestido Mrs en venta en tienda,fig_0,1
1,987137,75369,zRHRV9ym,Portátil Will,246.61,18,Artículo Portátil Will en venta en tienda,fig_1,1
2,800774,32684,fbIop2cv,Cocina Forward,209.73,6,Artículo Cocina Forward en venta en tienda,fig_2,1
3,524296,97521,SJTH7M23,Anillo Sell,637.04,65,Artículo Anillo Sell en venta en tienda,fig_3,1
4,163850,39582,RfF28wKl,Pantalón Article,504.89,5,Artículo Pantalón Article en venta en tienda,fig_4,1


In [21]:
sql_query = 'SELECT * FROM categoria;'
df_categoria = pd.read_sql(sql_query, postgres_driver)
df_categoria = df_categoria.rename(columns={'nombre': 'nombre_categoria', 'descripcion': 'descripcion_categoria', 'estado': 'estado_categoria'})
df_categoria.head()

,idcategoria,nombre_categoria,descripcion_categoria,estado_categoria
0,39582,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
1,82016,Calzado,"Incluye zapatos formales, tenis, botas, sandal...",1
2,51470,Juguetes y Juegos,Una amplia gama de juguetes para ninos de toda...,1
3,75369,Electronicos,"Desde telefonos moviles, computadoras portatil...",1
4,20934,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1


In [22]:
df_articulo = df_articulo.merge(df_categoria, how='inner', on='idcategoria')
dimArticulo = df_articulo.drop(['idcategoria'], axis=1)
dimArticulo.head()

,idarticulo,codigo,nombre_articulo,precio_venta,stock,descripcion_articulo,imagen,estado_articulo,nombre_categoria,descripcion_categoria,estado_categoria
0,403456,wkzOzRUc,Vestido Mrs,420.41,48,Artículo Vestido Mrs en venta en tienda,fig_0,1,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
1,163850,RfF28wKl,Pantalón Article,504.89,5,Artículo Pantalón Article en venta en tienda,fig_4,1,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
2,565277,H7ynkc4x,Pantalón Road,46.56,35,Artículo Pantalón Road en venta en tienda,fig_14,1,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
3,878635,QGNRRKUu,Suéter Out,595.43,9,Artículo Suéter Out en venta en tienda,fig_22,1,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
4,854061,b6qGemma,Suéter Door,40.72,82,Artículo Suéter Door en venta en tienda,fig_26,1,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1


#### Dimensión Venta


In [23]:
sql_query = 'SELECT * FROM venta;'
df_venta = pd.read_sql(sql_query, postgres_driver)
df_venta = df_venta.rename(columns={'estado': 'estado_venta'})
df_venta.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta
0,440324,161533493,5132316,Ticket de Venta,9392128,60010496,2017-04-25,33.73,30667.30,En proceso
1,886789,827908426,1502402,Boleta de Venta,6043652,91457537,2016-09-28,40.77,69801.72,Cancelada
2,747524,452401638,6895673,Ticket de Venta,2856965,67110914,2023-02-08,7.35,24363.43,Reembolsada
3,718855,516310396,3812010,Nota de Débito,5607430,75182086,2018-01-16,92.66,39454.25,Cancelada
4,882698,976254212,5491956,Nota de Débito,8556551,67856392,2023-06-27,57.23,61716.32,Cancelada


In [24]:
sql_query = 'SELECT * FROM persona'
df_cliente = pd.read_sql(sql_query, postgres_driver)
df_cliente = df_cliente[df_cliente['tipo_persona']=='cliente']
df_cliente = df_cliente.rename(columns={'idpersona': 'idcliente', 'nombre': 'nombre_cliente', 'tipo_documento': 'tipo_documento_cliente', 'num_documento': 'num_documento_cliente','direccion': 'direccion_cliente', 'telefono': 'telefono_cliente', 'email': 'email_cliente'})
df_cliente.head()

,idcliente,tipo_persona,nombre_cliente,tipo_documento_cliente,num_documento_cliente,direccion_cliente,telefono_cliente,email_cliente
0,463292416,cliente,Desiree Diaz,Pasaporte,147682912346113,"9238 Owens Land Suite 734 North Gabriel, NH 77058",+1-686-8,wongronald@hotmail.com
6,444735508,cliente,Michelle Henderson,DNI,193064418697232,"65789 Dennis Park Tylerstad, FL 81223",(940)380,tgibson@yahoo.com
7,488015896,cliente,Mr. James Stewart,DNI,168386493718550,"462 Koch Glen Port Michellefort, MT 33005",28880780,angela95@gmail.com
8,716460059,cliente,Nathaniel Barry,Pasaporte,159245248190492,"60490 Kathryn Cliff Suite 043 West Robert, CA ...",91736717,ronaldfoster@gmail.com
11,614416411,cliente,Amanda Hurst,NIE,155071137714219,"6624 Tiffany Junctions Lorimouth, VT 91011",001-586-,qharvey@gmail.com


In [25]:
df_venta_cliente = df_venta.merge(df_cliente, how='inner', on='idcliente')
df_venta_cliente.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta,tipo_persona,nombre_cliente,tipo_documento_cliente,num_documento_cliente,direccion_cliente,telefono_cliente,email_cliente
0,747524,452401638,6895673,Ticket de Venta,2856965,67110914,2023-02-08,7.35,24363.43,Reembolsada,cliente,Christopher Powers,NIE,155851439253989,"452 Wise Hollow Apt. 949 Hamptontown, MS 74863",911.689.,yho@gmail.com
1,718855,516310396,3812010,Nota de Débito,5607430,75182086,2018-01-16,92.66,39454.25,Cancelada,cliente,Monica Charles,Tarjeta Residencia,147774982911315,"430 Cain Forges Suite 877 Josephchester, VI 87388",(407)812,chadwheeler@yahoo.com
2,687469,516310396,8745434,Nota de Crédito,6477087,80168172,2022-01-02,87.96,6022.90,En proceso,cliente,Monica Charles,Tarjeta Residencia,147774982911315,"430 Cain Forges Suite 877 Josephchester, VI 87388",(407)812,chadwheeler@yahoo.com
3,772111,475790643,8124808,Boleta de Venta,2994197,75370512,2019-08-21,26.39,24372.48,Completada,cliente,Cindy Williams,NIE,139396824773914,"PSC 0310, Box 5624 APO AP 78701",647.995.,xwilson@yahoo.com
4,593937,955196919,6366513,Ticket de Venta,1034269,79214611,2015-06-14,5.72,40412.01,En proceso,cliente,Brenda Macdonald,NIE,160197191265706,"12028 Brandy Route Apt. 480 Lake Tylerside, PA...",+1-556-6,rguerra@yahoo.com


In [26]:
sql_query = 'SELECT * FROM usuario;'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario = df_usuario.rename(columns={'nombre': 'nombre_usuario', 'tipo_documento': 'tipo_documento_usuario', 'num_documento': 'num_documento_usuario','direccion': 'direccion_usuario', 'telefono': 'telefono_usuario', 'email': 'email_usuario'})
df_usuario.head()

,idusuario,idrol,nombre_usuario,tipo_documento_usuario,num_documento_usuario,direccion_usuario,telefono_usuario,email_usuario,clave,estado
0,7868418,10004,Luis Warren,Pasaporte,152338746871809,"3080 Travis Freeway Suite 836 Yorkshire, PA 42313",464-312-,lance31@gmail.com,"[b'd', b'9', b'f', b'1', b'5', b'b', b'a', b'9...",0
1,8210435,10004,Kristen Perez,Pasaporte,114070976995330,"854 Rush Ways East Melvinborough, NM 67431",+1-965-6,marcusjones@yahoo.com,"[b'1', b'b', b'1', b'0', b'f', b'f', b'c', b'6...",1
2,6367235,10002,Andrea Carney,DNI,146279382966274,USCGC Campos FPO AE 16169,001-468-,gainesgregory@yahoo.com,"[b'f', b'c', b'c', b'f', b'f', b'9', b'8', b'2...",1
3,1214471,10003,Lori Morrison,NIE,121915984308230,"081 Jill Run Lake Stephanieborough, MT 86103",001-274-,nwilson@hotmail.com,"[b'7', b'7', b'6', b'b', b'7', b'c', b'1', b'9...",1
4,6279175,10001,Jessica Chambers,Pasaporte,126574599972870,"5937 Sylvia Mountain South Shawn, SD 97049",+1-427-3,travismclean@gmail.com,"[b'4', b'1', b'3', b'8', b'b', b'd', b'd', b'7...",0


In [27]:
df_venta_cliente_usuario = df_venta_cliente.merge(df_usuario, how='inner', on='idusuario')
dimVenta = df_venta_cliente_usuario.drop(['idcliente', 'idusuario', 'idrol'], axis=1)
dimVenta.head()

,idventa,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta,tipo_persona,nombre_cliente,...,telefono_cliente,email_cliente,nombre_usuario,tipo_documento_usuario,num_documento_usuario,direccion_usuario,telefono_usuario,email_usuario,clave,estado
0,747524,Ticket de Venta,2856965,67110914,2023-02-08,7.35,24363.43,Reembolsada,cliente,Christopher Powers,...,911.689.,yho@gmail.com,Matthew Brown,NIE,105199439972415,"271 Timothy Crossing North Joseph, VA 16705",(268)240,jameswells@gmail.com,"[b'b', b'b', b'd', b'8', b'6', b'd', b'0', b'7...",1
1,476049,Nota de Débito,2730880,80659262,2021-08-21,90.76,21327.84,En proceso,cliente,Allison Jennings,...,001-384-,rodriguezjody@gmail.com,Matthew Brown,NIE,105199439972415,"271 Timothy Crossing North Joseph, VA 16705",(268)240,jameswells@gmail.com,"[b'b', b'b', b'd', b'8', b'6', b'd', b'0', b'7...",1
2,718855,Nota de Débito,5607430,75182086,2018-01-16,92.66,39454.25,Cancelada,cliente,Monica Charles,...,(407)812,chadwheeler@yahoo.com,Ruth Brown,NIE,128629693309651,"8620 Howell Cape South Travisstad, MS 77744",(716)989,lewisbrandy@hotmail.com,"[b'9', b'b', b'4', b'f', b'a', b'a', b'f', b'0...",0
3,742112,Boleta de Venta,4498143,99146405,2019-12-22,70.11,67210.84,Pendiente,cliente,Mrs. Jocelyn Sanders,...,(534)627,sarahjames@yahoo.com,Ruth Brown,NIE,128629693309651,"8620 Howell Cape South Travisstad, MS 77744",(716)989,lewisbrandy@hotmail.com,"[b'9', b'b', b'4', b'f', b'a', b'a', b'f', b'0...",0
4,687469,Nota de Crédito,6477087,80168172,2022-01-02,87.96,6022.90,En proceso,cliente,Monica Charles,...,(407)812,chadwheeler@yahoo.com,Kimberly Walker,NIE,181462814302696,"74484 Williams Village New Mark, IN 60405",+1-875-8,abrown@hotmail.com,"[b'9', b'7', b'3', b'2', b'd', b'b', b'6', b'1...",0


#### Dimensión Ingreso


In [28]:
sql_query = 'SELECT * FROM ingreso;'
df_ingreso = pd.read_sql(sql_query, postgres_driver)
df_ingreso.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,293646336,936021509,4783959,Ticket de Venta,7155714,None,2018-01-10,72.35,7016.36,Pendiente
1,820946950,853949599,4392714,Recibo,1759235,None,2016-05-31,40.71,83536.47,Cancelada
2,400994312,465096554,1142806,Nota de Débito,8587273,None,2018-10-26,94.24,14662.26,Cancelada
3,489381897,354924879,2332990,Factura,4892685,None,2022-07-09,68.01,26333.62,En proceso
4,946001930,276976903,6017611,Nota de Crédito,9670670,None,2018-02-12,95.31,68861.34,Completada


In [29]:
sql_query = 'SELECT * FROM persona;'
df_proveedor = pd.read_sql(sql_query, postgres_driver)
df_proveedor = df_proveedor[df_proveedor['tipo_persona']=='proveedor']
df_proveedor = df_proveedor.rename(columns={'idpersona': 'idproveedor', 'nombre': 'nombre_proveedor', 'tipo_documento': 'tipo_documento_proveedor', 'num_documento': 'num_documento_proveedor', 'direccion': 'direccion_proveedor', 'telefono': 'telefono_proveedor', 'email': 'email_proveedor'})
df_proveedor.head()

,idproveedor,tipo_persona,nombre_proveedor,tipo_documento_proveedor,num_documento_proveedor,direccion_proveedor,telefono_proveedor,email_proveedor
1,335015938,proveedor,Krista Allen,NIE,100989073293314,"438 James Fords Mariaport, VA 88487",(457)489,udiaz@hotmail.com
2,562499590,proveedor,Frank Fitzpatrick,Tarjeta Residencia,105173359417346,74414 Stephanie Place Suite 030 Lake Veronicac...,001-897-,rstone@hotmail.com
3,335101958,proveedor,Alicia Smith,NIE,124952882364420,"709 Mary Alley North Nicole, KY 44947",001-996-,eric43@hotmail.com
4,910546963,proveedor,Joshua Garrett,Pasaporte,115498004649989,"063 Gomez Causeway Lake Frank, AK 02159",707.966.,stonejeffery@gmail.com
5,838610964,proveedor,Ricardo Larson,DNI,179026619291649,"85593 Estrada Corner Suite 665 Peterston, TN 6...",001-477-,christopher87@hotmail.com


In [30]:
df_ingreso_proveedor = df_ingreso.merge(df_proveedor, how='inner', on='idproveedor')
df_ingreso_proveedor.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado,tipo_persona,nombre_proveedor,tipo_documento_proveedor,num_documento_proveedor,direccion_proveedor,telefono_proveedor,email_proveedor
0,820946950,853949599,4392714,Recibo,1759235,None,2016-05-31,40.71,83536.47,Cancelada,proveedor,Elizabeth White,Pasaporte,164454832284798,"6313 Bailey Springs Apt. 251 New Maryfurt, GA ...",+1-270-7,jason03@yahoo.com
1,300224655,853949599,7004573,Ticket de Venta,7372945,None,2020-01-05,22.01,93671.82,Pendiente,proveedor,Elizabeth White,Pasaporte,164454832284798,"6313 Bailey Springs Apt. 251 New Maryfurt, GA ...",+1-270-7,jason03@yahoo.com
2,506528209,853949599,1244310,Factura,7707144,None,2021-07-16,56.37,45546.03,En proceso,proveedor,Elizabeth White,Pasaporte,164454832284798,"6313 Bailey Springs Apt. 251 New Maryfurt, GA ...",+1-270-7,jason03@yahoo.com
3,170641419,364928153,3191413,Nota de Crédito,7145488,None,2015-11-22,91.03,310.80,Cancelada,proveedor,Jerome Miller,DNI,110967075994745,"40850 Cindy Fork Gallowayland, MP 12351",525-460-,john15@gmail.com
4,186235616,364928153,2748020,Ticket de Venta,4451087,None,2020-04-05,57.81,63320.94,Cancelada,proveedor,Jerome Miller,DNI,110967075994745,"40850 Cindy Fork Gallowayland, MP 12351",525-460-,john15@gmail.com


In [31]:
sql_query = 'SELECT * FROM usuario'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_proveedor = df_proveedor.rename(columns={'nombre': 'nombre_usuario', 'tipo_documento': 'tipo_documento_usuario', 'num_documento': 'num_documento_usuario', 'direccion': 'direccion_usuario', 'telefono': 'telefono_usuario', 'email': 'email_usuario'})
df_usuario.head()

,idusuario,idrol,nombre,tipo_documento,num_documento,direccion,telefono,email,clave,estado
0,7868418,10004,Luis Warren,Pasaporte,152338746871809,"3080 Travis Freeway Suite 836 Yorkshire, PA 42313",464-312-,lance31@gmail.com,"[b'd', b'9', b'f', b'1', b'5', b'b', b'a', b'9...",0
1,8210435,10004,Kristen Perez,Pasaporte,114070976995330,"854 Rush Ways East Melvinborough, NM 67431",+1-965-6,marcusjones@yahoo.com,"[b'1', b'b', b'1', b'0', b'f', b'f', b'c', b'6...",1
2,6367235,10002,Andrea Carney,DNI,146279382966274,USCGC Campos FPO AE 16169,001-468-,gainesgregory@yahoo.com,"[b'f', b'c', b'c', b'f', b'f', b'9', b'8', b'2...",1
3,1214471,10003,Lori Morrison,NIE,121915984308230,"081 Jill Run Lake Stephanieborough, MT 86103",001-274-,nwilson@hotmail.com,"[b'7', b'7', b'6', b'b', b'7', b'c', b'1', b'9...",1
4,6279175,10001,Jessica Chambers,Pasaporte,126574599972870,"5937 Sylvia Mountain South Shawn, SD 97049",+1-427-3,travismclean@gmail.com,"[b'4', b'1', b'3', b'8', b'b', b'd', b'd', b'7...",0


In [32]:
dimIngreso = df_ingreso_proveedor.merge(df_usuario, how='inner', on='idusuario')
dimIngreso.drop(['idproveedor', 'idusuario', 'idrol'], axis=1,  inplace=True)
dimIngreso.head()

,idingreso,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_x,tipo_persona,nombre_proveedor,...,telefono_proveedor,email_proveedor,nombre,tipo_documento,num_documento,direccion,telefono,email,clave,estado_y
0,820946950,Recibo,1759235,None,2016-05-31,40.71,83536.47,Cancelada,proveedor,Elizabeth White,...,+1-270-7,jason03@yahoo.com,Katherine Garcia,Pasaporte,190842568709907,"71047 Antonio Bypass Joeborough, WI 54705",(527)361,julie06@gmail.com,"[b'd', b'3', b'7', b'5', b'b', b'2', b'a', b'0...",0
1,940750154,Factura,3583267,None,2016-08-28,74.12,51546.08,Pendiente,proveedor,Kaitlyn Ryan,...,753.898.,rodney10@yahoo.com,Katherine Garcia,Pasaporte,190842568709907,"71047 Antonio Bypass Joeborough, WI 54705",(527)361,julie06@gmail.com,"[b'd', b'3', b'7', b'5', b'b', b'2', b'a', b'0...",0
2,300224655,Ticket de Venta,7372945,None,2020-01-05,22.01,93671.82,Pendiente,proveedor,Elizabeth White,...,+1-270-7,jason03@yahoo.com,John Williams,DNI,109136830015900,"26916 Dalton Crescent North Josephstad, WA 75499",001-275-,swood@hotmail.com,"[b'7', b'7', b'e', b'4', b'7', b'f', b'b', b'e...",1
3,439912004,Recibo,4034094,None,2021-02-01,85.56,62874.50,En proceso,proveedor,Nicole Soto,...,224-398-,benjamin95@yahoo.com,John Williams,DNI,109136830015900,"26916 Dalton Crescent North Josephstad, WA 75499",001-275-,swood@hotmail.com,"[b'7', b'7', b'e', b'4', b'7', b'f', b'b', b'e...",1
4,506528209,Factura,7707144,None,2021-07-16,56.37,45546.03,En proceso,proveedor,Elizabeth White,...,+1-270-7,jason03@yahoo.com,Kayla Hamilton,NIE,116011964916925,"PSC 2265, Box 5100 APO AP 27739",696-803-,vcortez@yahoo.com,"[b'5', b'0', b'f', b'c', b'4', b'a', b'f', b'9...",1


#### Dimensión Detalle Venta

In [33]:
sql_query = 'SELECT * FROM detalle_venta;'
df_detalle_venta = pd.read_sql(sql_query, postgres_driver)
df_detalle_venta.head()

,iddetalle_venta,idventa,idarticulo,cantidad,precio,descuento
0,4096006,355197,948995,65,573.12,357.44
1,4816907,863190,663080,24,82.49,945.63
2,9699343,741402,884690,41,271.86,957.04
3,9338899,848401,620664,61,697.37,510.88
4,4718616,874179,332521,42,227.73,345.11


In [34]:
sql_query = 'SELECT * FROM venta;'
df_venta = pd.read_sql(sql_query, postgres_driver)
df_venta = df_venta.rename(columns={'estado': 'estado_venta'})
df_venta.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta
0,440324,161533493,5132316,Ticket de Venta,9392128,60010496,2017-04-25,33.73,30667.30,En proceso
1,886789,827908426,1502402,Boleta de Venta,6043652,91457537,2016-09-28,40.77,69801.72,Cancelada
2,747524,452401638,6895673,Ticket de Venta,2856965,67110914,2023-02-08,7.35,24363.43,Reembolsada
3,718855,516310396,3812010,Nota de Débito,5607430,75182086,2018-01-16,92.66,39454.25,Cancelada
4,882698,976254212,5491956,Nota de Débito,8556551,67856392,2023-06-27,57.23,61716.32,Cancelada


In [35]:
df_detalle_venta_venta = df_detalle_venta.merge(df_venta, how='inner', on='idventa')
df_detalle_venta_venta.head()

,iddetalle_venta,idventa,idarticulo,cantidad,precio,descuento,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta
0,4096006,355197,948995,65,573.12,357.44,335101958,5813072,Ticket de Venta,7805814,56113962,2023-04-07,52.59,37535.91,Reembolsada
1,9833767,355197,340790,40,233.66,521.99,335101958,5813072,Ticket de Venta,7805814,56113962,2023-04-07,52.59,37535.91,Reembolsada
2,5396956,355197,227634,74,568.61,195.46,335101958,5813072,Ticket de Venta,7805814,56113962,2023-04-07,52.59,37535.91,Reembolsada
3,4809591,355197,387998,19,72.52,91.22,335101958,5813072,Ticket de Venta,7805814,56113962,2023-04-07,52.59,37535.91,Reembolsada
4,4713149,355197,285537,16,560.82,422.23,335101958,5813072,Ticket de Venta,7805814,56113962,2023-04-07,52.59,37535.91,Reembolsada


In [36]:
sql_query = 'SELECT *FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo = df_articulo.rename(columns={'estado': 'estado_articulo'})
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,403456,39582,wkzOzRUc,Vestido Mrs,420.41,48,Artículo Vestido Mrs en venta en tienda,fig_0,1
1,987137,75369,zRHRV9ym,Portátil Will,246.61,18,Artículo Portátil Will en venta en tienda,fig_1,1
2,800774,32684,fbIop2cv,Cocina Forward,209.73,6,Artículo Cocina Forward en venta en tienda,fig_2,1
3,524296,97521,SJTH7M23,Anillo Sell,637.04,65,Artículo Anillo Sell en venta en tienda,fig_3,1
4,163850,39582,RfF28wKl,Pantalón Article,504.89,5,Artículo Pantalón Article en venta en tienda,fig_4,1


In [37]:
dimDetalleVenta = df_detalle_venta_venta.merge(df_articulo, how='inner', on='idarticulo')
dimDetalleVenta.drop(['idventa', 'idarticulo', 'idcliente', 'idusuario'], axis=1,  inplace=True)
dimDetalleVenta.head()

,iddetalle_venta,cantidad,precio,descuento,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,4096006,65,573.12,357.44,Ticket de Venta,7805814,56113962,2023-04-07,52.59,37535.91,Reembolsada,40879,MZybOuGz,Perfume Public,573.12,7,Artículo Perfume Public en venta en tienda,fig_398,1
1,7549374,43,573.12,114.24,Ticket de Venta,3302553,88423546,2018-02-22,21.09,28862.28,En proceso,40879,MZybOuGz,Perfume Public,573.12,7,Artículo Perfume Public en venta en tienda,fig_398,1
2,5146385,27,573.12,683.13,Recibo,3827757,97994785,2016-10-11,92.32,55265.83,Completada,40879,MZybOuGz,Perfume Public,573.12,7,Artículo Perfume Public en venta en tienda,fig_398,1
3,5947354,26,573.12,825.77,Ticket de Venta,8606440,34222761,2020-12-13,55.19,20974.22,Reembolsada,40879,MZybOuGz,Perfume Public,573.12,7,Artículo Perfume Public en venta en tienda,fig_398,1
4,3973137,87,573.12,789.20,Nota de Débito,9532676,66856154,2015-11-17,70.69,61993.96,Cancelada,40879,MZybOuGz,Perfume Public,573.12,7,Artículo Perfume Public en venta en tienda,fig_398,1


#### Dimensión Detalle Ingreso


In [38]:
sql_query = 'SELECT * FROM detalle_ingreso;'
df_detalle_ingreso = pd.read_sql(sql_query, postgres_driver)
df_detalle_ingreso.head()

,iddetalle_ingreso,idingreso,idarticulo,cantidad,precio
0,554592256,619785452,120690,72,77.63
1,363057153,719109104,820244,52,748.48
2,541448192,166634843,681559,10,836.43
3,626491394,133733593,532432,77,47.44
4,755480581,822797124,464468,90,716.25


In [39]:
sql_query = 'SELECT * FROM ingreso;'
df_ingreso = pd.read_sql(sql_query, postgres_driver)
df_ingreso = df_ingreso.rename(columns={'estado': 'estado_ingreso'})
df_ingreso.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso
0,293646336,936021509,4783959,Ticket de Venta,7155714,None,2018-01-10,72.35,7016.36,Pendiente
1,820946950,853949599,4392714,Recibo,1759235,None,2016-05-31,40.71,83536.47,Cancelada
2,400994312,465096554,1142806,Nota de Débito,8587273,None,2018-10-26,94.24,14662.26,Cancelada
3,489381897,354924879,2332990,Factura,4892685,None,2022-07-09,68.01,26333.62,En proceso
4,946001930,276976903,6017611,Nota de Crédito,9670670,None,2018-02-12,95.31,68861.34,Completada


In [40]:
df_detalle_ingreso_ingreso = df_detalle_ingreso.merge(df_ingreso, how='inner', on='idingreso')
df_detalle_ingreso_ingreso.head()

,iddetalle_ingreso,idingreso,idarticulo,cantidad,precio,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso
0,554592256,619785452,120690,72,77.63,469523226,3730016,Factura,6632666,None,2017-10-24,53.38,5620.75,Reembolsada
1,178550127,619785452,120690,24,77.63,469523226,3730016,Factura,6632666,None,2017-10-24,53.38,5620.75,Reembolsada
2,126244576,619785452,789798,90,541.54,469523226,3730016,Factura,6632666,None,2017-10-24,53.38,5620.75,Reembolsada
3,363057153,719109104,820244,52,748.48,972652341,7083973,Factura,5952512,None,2015-10-18,38.38,77033.56,Pendiente
4,468495634,719109104,192356,19,474.88,972652341,7083973,Factura,5952512,None,2015-10-18,38.38,77033.56,Pendiente


In [41]:
sql_query = 'SELECT * FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo = df_articulo.rename(columns={'estado': 'estado_articulo'})
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,403456,39582,wkzOzRUc,Vestido Mrs,420.41,48,Artículo Vestido Mrs en venta en tienda,fig_0,1
1,987137,75369,zRHRV9ym,Portátil Will,246.61,18,Artículo Portátil Will en venta en tienda,fig_1,1
2,800774,32684,fbIop2cv,Cocina Forward,209.73,6,Artículo Cocina Forward en venta en tienda,fig_2,1
3,524296,97521,SJTH7M23,Anillo Sell,637.04,65,Artículo Anillo Sell en venta en tienda,fig_3,1
4,163850,39582,RfF28wKl,Pantalón Article,504.89,5,Artículo Pantalón Article en venta en tienda,fig_4,1


In [42]:
dimDetalleIngreso = df_detalle_ingreso_ingreso.merge(df_articulo, how='inner', on='idarticulo')
dimDetalleIngreso.drop(['idingreso', 'idarticulo', 'idproveedor', 'idusuario'], axis=1,  inplace=True)
dimDetalleIngreso.head()

,iddetalle_ingreso,cantidad,precio,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,554592256,72,77.63,Factura,6632666,None,2017-10-24,53.38,5620.75,Reembolsada,32684,hz7oe5cS,Lámpara Opportunity,77.63,71,Artículo Lámpara Opportunity en venta en tienda,fig_928,1
1,178550127,24,77.63,Factura,6632666,None,2017-10-24,53.38,5620.75,Reembolsada,32684,hz7oe5cS,Lámpara Opportunity,77.63,71,Artículo Lámpara Opportunity en venta en tienda,fig_928,1
2,629893402,17,77.63,Factura,2982792,None,2015-07-16,90.14,96204.87,Cancelada,32684,hz7oe5cS,Lámpara Opportunity,77.63,71,Artículo Lámpara Opportunity en venta en tienda,fig_928,1
3,774386291,81,77.63,Factura,8349831,None,2023-10-13,60.65,91260.35,En proceso,32684,hz7oe5cS,Lámpara Opportunity,77.63,71,Artículo Lámpara Opportunity en venta en tienda,fig_928,1
4,126244576,90,541.54,Factura,6632666,None,2017-10-24,53.38,5620.75,Reembolsada,40879,pc6KNQxt,Maquillaje Option,541.54,45,Artículo Maquillaje Option en venta en tienda,fig_653,1


#### Dimensión Rol


In [43]:
sql_query = 'SELECT * FROM rol;'
dimRol = pd.read_sql(sql_query, postgres_driver)
dimRol.head()

,idrol,nombre,descripcion,estado
0,10001,Administrador,Este rol tiene acceso completo al sistema y pu...,1
1,10002,Moderador,Este rol tiene permisos para moderar contenido...,1
2,10003,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,10004,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
4,10005,Invitado,Este rol tiene acceso limitado y solo puede ve...,1


#### Dimensión Categoría


In [44]:
sql_query = 'SELECT * FROM categoria;'
dimCategoria = pd.read_sql(sql_query, postgres_driver)
dimCategoria.head()

,idcategoria,nombre,descripcion,estado
0,39582,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
1,82016,Calzado,"Incluye zapatos formales, tenis, botas, sandal...",1
2,51470,Juguetes y Juegos,Una amplia gama de juguetes para ninos de toda...,1
3,75369,Electronicos,"Desde telefonos moviles, computadoras portatil...",1
4,20934,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1


#### Dimensión de Tiempo


In [45]:
sql_query = 'SELECT fecha FROM venta;'
dimFecha = pd.read_sql(sql_query, postgres_driver)
dimFecha.head()

,fecha
0,2017-04-25
1,2016-09-28
2,2023-02-08
3,2018-01-16
4,2023-06-27


In [46]:
dimFecha['year'] = pd.DatetimeIndex(dimFecha['fecha']).year
dimFecha['month'] = pd.DatetimeIndex(dimFecha['fecha']).month
dimFecha['quarter'] = pd.DatetimeIndex(dimFecha['fecha']).quarter
dimFecha['day'] = pd.DatetimeIndex(dimFecha['fecha']).day
dimFecha['week'] = pd.DatetimeIndex(dimFecha['fecha']).week
dimFecha['dayofweek'] = pd.DatetimeIndex(dimFecha['fecha']).dayofweek
#dimFechaHora['hour'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).hour
#dimFechaHora['minute'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).minute
dimFecha['is_weekend'] = dimFecha['dayofweek'].apply(lambda x: 1 if x > 5 else 0)
dimFecha.head()

/var/folders/t4/ltm_f1s92xd8rkv3v0nc5ll80000gn/T/ipykernel_22790/1606713047.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dimFecha['week'] = pd.DatetimeIndex(dimFecha['fecha']).week


,fecha,year,month,quarter,day,week,dayofweek,is_weekend
0,2017-04-25,2017,4,2,25,17,1,0
1,2016-09-28,2016,9,3,28,39,2,0
2,2023-02-08,2023,2,1,8,6,2,0
3,2018-01-16,2018,1,1,16,3,1,0
4,2023-06-27,2023,6,2,27,26,1,0


In [47]:
dimFecha['id_fecha'] = dimFecha['year'].astype(str) + dimFecha['month'].astype(str)
dimFecha['id_fecha'] = dimFecha['id_fecha'].astype(str) + dimFecha['day'].astype(str)
#dimFecha['id_fecha'] = dimFecha['id_fecha'].astype(str) + dimFechaHora['hour'].astype(str)
#dimFechaHora['id_fecha'] = dimFechaHora['id_fecha'].astype(str) + dimFechaHora['minute'].astype(str)
dimFecha.drop_duplicates(inplace=True)
dimFecha.head()

,fecha,year,month,quarter,day,week,dayofweek,is_weekend,id_fecha
0,2017-04-25,2017,4,2,25,17,1,0,2017425
1,2016-09-28,2016,9,3,28,39,2,0,2016928
2,2023-02-08,2023,2,1,8,6,2,0,202328
3,2018-01-16,2018,1,1,16,3,1,0,2018116
4,2023-06-27,2023,6,2,27,26,1,0,2023627


#### Tabla de Hechos


In [48]:
sql_query = '''SELECT idventa, fecha, idcliente,
                              idusuario, tipo_comprobante, serie_comprobante, num_comprobante, impuesto, total,
                              estado FROM venta;'''
df_factTable = pd.read_sql(sql_query, postgres_driver)
df_factTable.head()

,idventa,fecha,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,impuesto,total,estado
0,440324,2017-04-25,161533493,5132316,Ticket de Venta,9392128,60010496,33.73,30667.30,En proceso
1,886789,2016-09-28,827908426,1502402,Boleta de Venta,6043652,91457537,40.77,69801.72,Cancelada
2,747524,2023-02-08,452401638,6895673,Ticket de Venta,2856965,67110914,7.35,24363.43,Reembolsada
3,718855,2018-01-16,516310396,3812010,Nota de Débito,5607430,75182086,92.66,39454.25,Cancelada
4,882698,2023-06-27,976254212,5491956,Nota de Débito,8556551,67856392,57.23,61716.32,Cancelada
